<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Task" data-toc-modified-id="1.-Task-1">1. Task</a></span></li><li><span><a href="#2.-Data" data-toc-modified-id="2.-Data-2">2. Data</a></span></li><li><span><a href="#3.-Notebook" data-toc-modified-id="3.-Notebook-3">3. Notebook</a></span></li><li><span><a href="#4.-Conclusion" data-toc-modified-id="4.-Conclusion-4">4. Conclusion</a></span></li><li><span><a href="#5.-Appendix" data-toc-modified-id="5.-Appendix-5">5. Appendix</a></span></li></ul></div>

------------------------------------------------------------------
 # Geospatial Data Exercise
------------------------------------------------------------------

 This is an exercise notebook for the fifth lesson of the kaggle course
 ["Geospatial Analysis"](https://www.kaggle.com/learn/geospatial-analysis)
 offered by Alexis Cook and Jessica Li. The main goal of the lesson is
 to get used to __Proximity Analysis__, using `geopandas` methods such as
 `.distance`. We also learn how to use
 `.unary_union` to connect multiple `POLYGON`s into one.

## 1. Task

   Every day someone injured in New York City in a car accident.
   If an ambulance can quickly rush into a nearby hospital with a patient
   is a matter of life and death. We will review the records of daily car
   crashes in New York City and the locations of hospitals there.

 1. Find out if there is any vulnerable districts where
    it takes particularly long to transport the injured to a hospital.
 2. Create a recommender system to tell ambulance drivers
    to which hospital (the nearest) they should transport the injured.

## 2. Data

 1. Daily records of car crashes in New York City.

 2. Locations of hospitals in New York City.

 3. General underlying  map.

## 3. Notebook

Import packages.

In [1]:
import folium
import numpy as np
from folium import Marker, GeoJson
from folium.plugins import HeatMap
import pandas as pd
import geopandas as gpd
import plotly.graph_objs as go
from kaggle_geospatial.kgsp import *
from datetime import datetime
import os
from pathlib import Path

Set up some directories.

In [2]:
CWD = '/Users/meg/git6/ny_hospitals/'
DATA_DIR = '../input/geospatial-learn-course-data/'
KAGGLE_DIR = 'alexisbcook/geospatial-learn-course-data'
GEO_DIR = 'geospatial-learn-course-data'

os.chdir(CWD)

set_cwd(CWD)
set_data_dir(DATA_DIR, KAGGLE_DIR, GEO_DIR, CWD)
show_whole_dataframe(True)

Read the daily records of car crashes in New York City.

In [3]:
collisions_dir = DATA_DIR + \
    'NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions/'

collisions = gpd.read_file(
    collisions_dir + 'NYPD_Motor_Vehicle_Collisions.shp',
    parse_dates=['DATE', 'TIME'])

print(collisions.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 261905 entries, 0 to 261904
Data columns (total 30 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   DATE        261905 non-null  object  
 1   TIME        261905 non-null  object  
 2   BOROUGH     199243 non-null  object  
 3   ZIP CODE    199200 non-null  object  
 4   LATITUDE    261905 non-null  float64 
 5   LONGITUDE   261905 non-null  float64 
 6   LOCATION    261905 non-null  object  
 7   ON STREET   225789 non-null  object  
 8   CROSS STRE  185029 non-null  object  
 9   OFF STREET  23998 non-null   object  
 10  NUMBER OF   261905 non-null  float64 
 11  NUMBER O_1  261892 non-null  float64 
 12  NUMBER O_2  261905 non-null  int64   
 13  NUMBER O_3  261905 non-null  int64   
 14  NUMBER O_4  261905 non-null  int64   
 15  NUMBER O_5  261905 non-null  int64   
 16  NUMBER O_6  261905 non-null  int64   
 17  NUMBER O_7  261905 non-null  int64   
 18  CONTRIBUTI  2601

It looks like, `parse_dates` does not convert `dtype` of
 date and time to `datetime` from `object` (string).
 Do it separately.

In [4]:
collisions['DATE'] = pd.to_datetime(collisions['DATE'])
collisions['TIME'] = pd.to_datetime(collisions['TIME'])

Let us start with the record in 2019 only.

In [5]:
print(collisions['DATE'].min())
print(collisions['DATE'].max())
print(len(collisions))

collisions = collisions[collisions['DATE'] >=
                        datetime.strptime('2019/01/01', '%Y/%m/%d')]

print(collisions['DATE'].min())
print(collisions['DATE'].max())
print(len(collisions))
collisions.head(3)

2012-07-01 00:00:00
2019-07-30 00:00:00
261905
2019-01-01 00:00:00
2019-07-30 00:00:00
23517


,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,NUMBER OF,NUMBER O_1,NUMBER O_2,NUMBER O_3,NUMBER O_4,NUMBER O_5,NUMBER O_6,NUMBER O_7,CONTRIBUTI,CONTRIBU_1,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry
0,2019-07-30,2021-11-28 00:00:00,BRONX,10464,40.841100,-73.784960,"(40.8411, -73.78496)",None,None,121 PILOT STREET,1.0,0.0,0,0,0,0,1,0,Unspecified,Unspecified,Unspecified,None,None,4180045,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,POINT (1043750.211 245785.815)
1,2019-07-30,2021-11-28 00:10:00,QUEENS,11423,40.710827,-73.770660,"(40.710827, -73.77066)",JAMAICA AVENUE,188 STREET,None,1.0,0.0,0,0,0,0,1,0,Driver Inattention/Distraction,Unspecified,None,None,None,4180007,Sedan,Sedan,None,None,None,POINT (1047831.185 198333.171)
2,2019-07-30,2021-11-28 00:25:00,None,None,40.880318,-73.841286,"(40.880318, -73.841286)",BOSTON ROAD,None,None,1.0,0.0,0,0,0,0,1,0,Following Too Closely,Unspecified,None,None,None,4179575,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (1028139.293 260041.178)


Read the locations of hospitals in New York City.

In [6]:
hospitals_dir = DATA_DIR + 'nyu_2451_34494/nyu_2451_34494/'
hospitals = gpd.read_file(hospitals_dir + 'nyu_2451_34494.shp')

print(hospitals.info())
print(hospitals.shape)
hospitals.head(3)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   id         62 non-null     object  
 1   name       62 non-null     object  
 2   address    62 non-null     object  
 3   zip        62 non-null     object  
 4   factype    62 non-null     object  
 5   facname    62 non-null     object  
 6   capacity   60 non-null     object  
 7   capname    62 non-null     object  
 8   bcode      62 non-null     object  
 9   xcoord     62 non-null     float64 
 10  ycoord     62 non-null     float64 
 11  latitude   62 non-null     float64 
 12  longitude  62 non-null     float64 
 13  geometry   62 non-null     geometry
dtypes: float64(4), geometry(1), object(9)
memory usage: 6.9+ KB
None
(62, 14)


,id,name,address,zip,factype,facname,capacity,capname,bcode,xcoord,ycoord,latitude,longitude,geometry
0,317000001H1178,BRONX-LEBANON HOSPITAL CENTER - CONCOURSE DIVI...,1650 Grand Concourse,10457,3102,Hospital,415,Beds,36005,1008872.0,246596.0,40.843490,-73.911010,POINT (1008872.000 246596.000)
1,317000001H1164,BRONX-LEBANON HOSPITAL CENTER - FULTON DIVISION,1276 Fulton Ave,10456,3102,Hospital,164,Beds,36005,1011044.0,242204.0,40.831429,-73.903178,POINT (1011044.000 242204.000)
2,317000011H1175,CALVARY HOSPITAL INC,1740-70 Eastchester Rd,10461,3102,Hospital,225,Beds,36005,1027505.0,248287.0,40.848060,-73.843656,POINT (1027505.000 248287.000)


In [7]:
# Create a heatmap to show how the car crashes in New York City distributed.
# First, set up the center of the map, tiles, and the initial zoom-factor.

center = (collisions['LATITUDE'].mean(), collisions['LONGITUDE'].mean())
tiles = 'Stamen Terrain'
tiles = 'openstreetmap'
# tiles = 'cartodbpositron'
zoom = 12

m_1 = folium.Map(location=center, tiles=tiles, zoom_start=zoom)
HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']],
        min_opacity=0.1,
        radius=15).add_to(m_1)

In [8]:
embed_map(m_1, './html/m_1.html')

In [9]:
show_on_browser(m_1, CWD + './html/m_1b.html')

There are concentrations of car crashes in
 * Lower Manhattan
 * Brooklyn
 * The Bronx

in this order.

Let us overlay the locations of hospitals.

In [10]:
m_2 = folium.Map(location=center, tiles=tiles, zoom_start=zoom)
HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']],
        min_opacity=0.1,
        radius=15).add_to(m_2)
dump = [Marker(location=[r['latitude'], r['longitude']], tooltip=r['name'],
               popup=r['address']).add_to(m_2) for i, r in hospitals.iterrows()]

In [11]:
embed_map(m_2, './html/m_2.html')

In [12]:
show_on_browser(m_2, CWD + './html/m_2b.html')

Pick up the cases that the closest hospitals are more than 10 km away.

1. Add following columns to `collisions` table.
 - name, id and address of the nearest hospital.
 - distance to the nearest hospital.

2. Flag it when the nearest hospitals is more than 10 km away.
 Note that units of EPSG 2263 are meters.

In [13]:
print(hospitals.crs)
hospitals.crs == collisions.crs

epsg:2263


True

In [14]:
id_nearest_h = []
name_nearest_h = []
address_nearest_h = []
distance_nearest_h = []

for i, c in collisions.iterrows():
    distance = hospitals['geometry'].distance(c['geometry']).min()
    idx = hospitals['geometry'].distance(c['geometry']).idxmin()

    id_nearest_h.append(hospitals.iloc[idx]['id'])
    name_nearest_h.append(hospitals.iloc[idx]['name'])
    address_nearest_h.append(hospitals.iloc[idx]['address'])
    distance_nearest_h.append(distance)

collisions['id_NEAREST_H'] = id_nearest_h
collisions['NAME_NEAREST_H'] = name_nearest_h
collisions['ADDRESS_NEAREST_H'] = address_nearest_h
collisions['DISTANCE_NEAREST_H'] = distance_nearest_h

print(collisions.info())
collisions.head(3)

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 23517 entries, 0 to 23517
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                23517 non-null  datetime64[ns]
 1   TIME                23517 non-null  datetime64[ns]
 2   BOROUGH             15676 non-null  object        
 3   ZIP CODE            15669 non-null  object        
 4   LATITUDE            23517 non-null  float64       
 5   LONGITUDE           23517 non-null  float64       
 6   LOCATION            23517 non-null  object        
 7   ON STREET           19597 non-null  object        
 8   CROSS STRE          13262 non-null  object        
 9   OFF STREET          3916 non-null   object        
 10  NUMBER OF           23517 non-null  float64       
 11  NUMBER O_1          23517 non-null  float64       
 12  NUMBER O_2          23517 non-null  int64         
 13  NUMBER O_3          23517 non-null  in

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,NUMBER OF,NUMBER O_1,NUMBER O_2,NUMBER O_3,NUMBER O_4,NUMBER O_5,NUMBER O_6,NUMBER O_7,CONTRIBUTI,CONTRIBU_1,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry,id_NEAREST_H,NAME_NEAREST_H,ADDRESS_NEAREST_H,DISTANCE_NEAREST_H
0,2019-07-30,2021-11-28 00:00:00,BRONX,10464,40.841100,-73.784960,"(40.8411, -73.78496)",None,None,121 PILOT STREET,1.0,0.0,0,0,0,0,1,0,Unspecified,Unspecified,Unspecified,None,None,4180045,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,POINT (1043750.211 245785.815),317000011H1175,CALVARY HOSPITAL INC,1740-70 Eastchester Rd,16436.629798
1,2019-07-30,2021-11-28 00:10:00,QUEENS,11423,40.710827,-73.770660,"(40.710827, -73.77066)",JAMAICA AVENUE,188 STREET,None,1.0,0.0,0,0,0,0,1,0,Driver Inattention/Distraction,Unspecified,None,None,None,4180007,Sedan,Sedan,None,None,None,POINT (1047831.185 198333.171),317003007H1633,QUEENS HOSPITAL CENTER,82-68 164 St,10029.185965
2,2019-07-30,2021-11-28 00:25:00,None,None,40.880318,-73.841286,"(40.880318, -73.841286)",BOSTON ROAD,None,None,1.0,0.0,0,0,0,0,1,0,Following Too Closely,Unspecified,None,None,None,4179575,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (1028139.293 260041.178),317000006H1168,MONTEFIORE MEDICAL CENTER-WAKEFIELD HOSPITAL,600 E 233 St,7315.939667


How much is the fraction of car crashes
 that the nearest hospitals are
 more than 10 km away?

In [15]:
f_outside = (collisions['DISTANCE_NEAREST_H'] > 10 ** 4).mean()
print(
    f'\033[33mIn \033[96m{f_outside: 6.3f} \033[33m cases \
the nearest hospital is > 10 km away\033[0m')

In  0.161  cases the nearest hospital is > 10 km away


Find out which part of New York City,
 such cases often happen.
 Use `unary_union` that we learned in the previous lesson.

In [16]:
ten_km_buffer = gpd.GeoDataFrame(geometry=hospitals.buffer(10 ** 4))
ten_km_buffer = ten_km_buffer.to_crs(epsg=4326)
ten_km_union = ten_km_buffer.unary_union

# We do not need to add crs to `ten_km_union` as it is a single object
# `MultiPolygon`, not  a `GeoDataFrame`.

In [17]:
def style_function(x):

    return {'fillColor': 'salmon',
            'stroke': True, 'color': 'salmon', 'weight': 8,
            'fillOpacity': 0.2}  # 'dashArray' :  '5,5'


tiles = 'openstreetmap'
tiles = 'Stamen Terrain'
m_3 = folium.Map(location=center, tiles=tiles, zoom_start=zoom)

HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']],
        min_opacity=0.1,
        radius=15).add_to(m_3)

GeoJson(data=ten_km_union.__geo_interface__,
        style_function=style_function).add_to(m_3)

dump = [Marker([h['latitude'], h['longitude']],
               tooltip=h['name'],
               popup=h['address']).add_to(m_3)
        for i, h in hospitals.iterrows()]

folium.LatLngPopup().add_to(m_3)

In [18]:
embed_map(m_3, './html/m_3.html')

In [19]:
show_on_browser(m_3, CWD + './html/m_3b.html')

There are three sites that the new
 hospitals should be build.
 1. Along NY 27 at the northeast of the JFK airport.
 2. Interchange of NY 24 at the south of Belmont Park.
 3. East edge of Brooklyn at NY 27A.

Let us add two hospitals from our hypothesis.
 How much is the fraction of the car crashes now that happen
 outside of 10 km buffer of the NYC hospitals?

 From the reading of pop-up on the map,

In [20]:
h_1 = [-73.7691, 40.6679]
h_2 = [-73.8443, 40.6714]

new_hospitals = gpd.GeoDataFrame(geometry=gpd.points_from_xy(
    [h_1[0], h_2[0]], [h_1[1], h_2[1]], crs='epsg:4326'))

new_buffers = gpd.GeoDataFrame(
    geometry=new_hospitals.to_crs(epsg=2263).buffer(10 ** 4))

new_buffers = new_buffers.to_crs(epsg=4236)

In [21]:
def style_function2(x):

    return {'fillColor': 'maroon',
            'stroke': True, 'color': 'maroon', 'weight': 8,
            'fillOpacity': 0.2}  # 'dashArray' :  '5,5'


tiles = 'openstreetmap'
m_4 = folium.Map(location=center, tiles=tiles, zoom_start=zoom)

HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']],
        min_opacity=0.1,
        radius=15).add_to(m_4)

GeoJson(data=ten_km_union.__geo_interface__,
        style_function=style_function).add_to(m_4)

GeoJson(data=new_hospitals.__geo_interface__).add_to(m_4)

GeoJson(data=new_buffers.__geo_interface__,
        style_function=style_function2).add_to(m_4)

dump = [Marker([h['latitude'], h['longitude']],
               tooltip=h['name'],
               popup=h['address']).add_to(m_4)
        for i, h in hospitals.iterrows()]

folium.LatLngPopup().add_to(m_4)

In [22]:
embed_map(m_4, './html/m_4.html')

In [23]:
show_on_browser(m_4, CWD + './html/m_4b.html')

Looks good.

In [24]:
new_hospitals_ny = new_hospitals.to_crs(epsg=2263).copy()

all_hospitals = gpd.GeoDataFrame(pd.concat([hospitals['geometry'],
                                            new_hospitals_ny['geometry']], axis=0,
                                           ignore_index=True))

# It is enough to change the column name.

all_hospitals.rename(columns={0: 'geometry'}, inplace=True)
all_hospitals.crs = {'init': 'epsg:2263'}

distance_new = []
for i, c in collisions.iterrows():
    distance_new.append(
        all_hospitals['geometry'].distance(c['geometry']).min())

f2_outside = (np.array(distance_new) > 10 ** 4).mean()

print(
    f'\033[33mIn \033[96m{f2_outside: 6.3f} \033[33m cases \
the nearest hospital is > 10 km away\033[0m')

/Users/meg/opt/anaconda3/envs/k3/lib/python3.8/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In  0.099  cases the nearest hospital is > 10 km away


## 4. Conclusion

 We tried both combinations above, i.e., (1,2) and (1,3).
 We found (1,3) is more effective to reduce the fraction
 of car crashes outside 10 km of a hospital.
 Again the proposed sites are

In [25]:
h_1 = [-73.7691, 40.6679]
h_2 = [-73.8443, 40.6714]

## 5. Appendix

 Write a function that takes longitude and latitude of a car crash,
 and returns the name and the address of the nearest hospital.

In [26]:
def nearest_hospital(z, hospitals) -> gpd.GeoDataFrame:
    '''
    z : gpd.GeoDataFrame.  
        z['geometry'] contains the longitude and the latitude 
        of the place where the car crash happened. 

      hospitals : gpd.GeoDataFrame. A list of locations and other 
    informations (name, address, etc.) of the hospitals 
    in New York City.

    '''
    z = z.to_crs(epsg=2263)
    z = z.iloc[0]

    idx = hospitals['geometry'].distance(z['geometry']).idxmin()
    distance = hospitals['geometry'].distance(z['geometry']).min() / 10**3

    print(f'DISTANCE : \033[96m {distance:5.1f} \033[0mkm')

    return hospitals.iloc[idx][['name', 'address', 'longitude', 'latitude']]

Test.

In [27]:
# Imaginative crash location.
z = gpd.GeoDataFrame(geometry=gpd.points_from_xy(
    [-73.9683], [40.7937], crs='epsg:4326'))

nearest_hospital(z, hospitals)

DISTANCE :    4.4 km


name         MOUNT SINAI HOSPITAL
address       1 Gustave L Levy Pl
longitude              -73.953266
latitude                40.789875
Name: 37, dtype: object

END